# Install requirements



In [72]:
pip install tenseal sentence_transformers

In [73]:
import tenseal as ts # pip install tenseal
import base64
from sentence_transformers import SentenceTransformer

#Setup Embedding Model

In [74]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [75]:
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "I watched the sunset over the ocean.",
    "Artificial intelligence is transforming industries.",
    "The library was quiet and smelled like old books.",
    "He dreamed of traveling to distant galaxies.",
    "Innovation drives progress in the tech world.",
    "The chef prepared a delicious meal for the guests.",
    "Climate change poses a significant threat to global biodiversity.",
    "The athlete trained rigorously for the upcoming marathon.",
    "Music has the power to evoke deep emotional responses."
]

# Get and Store Sentence Embeddings

In [76]:
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.04393356  0.05893442  0.04817836 ...  0.05216276  0.05610649
   0.10206394]
 [ 0.02740657  0.0430141   0.03891347 ... -0.01506037 -0.08190976
   0.03566786]
 [ 0.03608515 -0.02447893  0.05895455 ... -0.08262778  0.07626031
  -0.02810953]
 ...
 [-0.04316908  0.12324493  0.09774672 ... -0.13736206 -0.0542914
  -0.02212726]
 [-0.01195203  0.08340716 -0.03579247 ... -0.0014969   0.00464318
  -0.00069307]
 [ 0.05495496 -0.0110149   0.09608352 ...  0.05075204  0.01033213
  -0.03917952]]


# Get Query Embedding

In [129]:
query_text = "Artificial intelligence is transforming industries."
query_embedding = model.encode([query_text])
print(query_embedding)

[[ 3.60851325e-02 -2.44789366e-02  5.89545183e-02 -3.21429246e-03
   4.38118353e-02  1.85996927e-02 -1.78743470e-02 -4.63144248e-03
  -1.62027348e-02 -3.14670093e-02 -4.96116020e-02  8.08451101e-02
  -2.00940073e-02 -1.92342326e-03 -2.43032053e-02  3.15106027e-02
  -5.34295104e-02  5.58675849e-04 -1.01851113e-01 -9.03230533e-02
   3.34103368e-02  4.79379017e-03 -4.64723632e-02 -3.10845505e-02
   3.30479331e-02  5.18943928e-02  1.34106306e-02 -1.10997874e-02
  -4.43072878e-02 -5.49910590e-02 -4.65399697e-02  7.50129446e-02
   4.76049818e-02  4.27132770e-02  3.35510597e-02  7.91164339e-02
   2.42569391e-02  1.74476560e-02  8.09293836e-02 -9.92202852e-03
   3.46010067e-02 -1.11723728e-01 -3.14392820e-02 -2.51279734e-02
   6.11408651e-02  7.95261264e-02  5.71230426e-03 -2.10794471e-02
   2.01567896e-02  2.59161312e-02 -1.67493403e-01 -2.59155612e-02
   5.32490537e-02  4.23922017e-03 -5.21578267e-02  2.88533941e-02
   5.73436171e-02 -4.07534800e-02  7.47103384e-03 -1.51069853e-02
   3.81493

# Setup CKKS Homomorphic Encryption

In [130]:
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree = 8192,
            coeff_mod_bit_sizes = [60, 40, 40, 60]
          )

context.generate_galois_keys()
context.global_scale = 2**40

In [131]:
secret_context = context.serialize(save_secret_key = True)

In [132]:
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()

In [133]:
context = ts.context_from(secret_context)

# Encrypt All Embeddings

In [134]:
enc_queryvec = ts.ckks_vector(context, query_embedding[0].tolist())
print(enc_queryvec.decrypt())

[0.03608513113383832, -0.024478936326003738, 0.05895451790581528, -0.0032142929050931815, 0.04381183499707305, 0.018599692621104502, -0.017874346570614492, -0.004631440917279373, -0.016202734363539348, -0.031467009233617546, -0.04961160351448522, 0.08084510803796882, -0.02009400762059079, -0.0019234260663707411, -0.024303204345803835, 0.031510603936170584, -0.05342950873636152, 0.0005586758079435469, -0.10185111273609469, -0.09032305359975673, 0.03341033603229296, 0.004793790268919117, -0.04647236332768068, -0.031084550122712665, 0.03304793205466674, 0.05189439567957291, 0.013410629609135865, -0.01109978887748959, -0.044307287860951845, -0.0549910577883782, -0.04653997030347586, 0.07501294615342324, 0.047604983556611476, 0.042713277341152586, 0.03355106027282079, 0.07911643331254395, 0.02425693852694871, 0.017447656471013064, 0.0809293833356218, -0.009922029562551608, 0.03460100506328996, -0.1117237278517132, -0.03143928126095097, -0.025127973260460016, 0.06114086579738352, 0.079526125

Measure Eucledian Distance

In [135]:
euclidean_difference_ranking = []
for i in range (len(sentences)):
  enc_sentence = ts.ckks_vector(context, embeddings[i].tolist())
  euclidean_squared =   enc_queryvec - enc_sentence
  euclidean_difference_ranking.append({sentences[i]:abs(euclidean_squared.decrypt()[0])})

# Present Results

In [136]:
search_results = sorted(euclidean_difference_ranking, key=lambda x: list(x.values())[0])
for item in search_results:
  print(item)
  print()

{'Artificial intelligence is transforming industries.': 2.1452503881992105e-08}

{'The quick brown fox jumps over the lazy dog.': 0.007848430681761193}

{'The library was quiet and smelled like old books.': 0.007859144698194348}

{'I watched the sunset over the ocean.': 0.008678558683020421}

{'Music has the power to evoke deep emotional responses.': 0.018869825973224112}

{'Innovation drives progress in the tech world.': 0.023792035812631472}

{'The chef prepared a delicious meal for the guests.': 0.046898367326590855}

{'The athlete trained rigorously for the upcoming marathon.': 0.0480371589371672}

{'He dreamed of traveling to distant galaxies.': 0.051423703642053065}

{'Climate change poses a significant threat to global biodiversity.': 0.07925421433619254}

